In [1]:
device='cuda'

## Import Packages

In [2]:
!pip install coremltools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00


In [3]:
import os
import json
import numpy as np
from google.colab import drive
from torchvision.models import vit_b_32, ViT_B_32_Weights
from transformers import BertTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import coremltools as ct

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


## Convert Encoder

1. Create a wrapped model

In [21]:
weights = ViT_B_32_Weights.DEFAULT
encoder = vit_b_32(weights=weights)
encoder.heads = torch.nn.Identity()
preprocess = weights.transforms()

In [15]:
preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class WrappedEncoder(nn.Module):
  def __init__(self, encoder):
    super(WrappedEncoder, self).__init__()
    self.encoder = encoder



  def forward(self, x):
    x = preprocess(x)
    output = self.encoder(x)
    output = output.repeat(32, 1)
    output = output.unsqueeze(0)
    return output

iOS_Encoder = WrappedEncoder(encoder)

2. Trace the wrapped model with an example input

In [16]:
traced_encoder_model = torch.jit.trace(iOS_Encoder, torch.rand(1, 3, 224, 224))

3. Convert the traced model into a CoreML model

In [19]:
encoder_model = ct.convert(
    traced_encoder_model,
    inputs=[ct.ImageType(shape=(1, 3, 224, 224), scale=1/255)],
)

encoder_model.save("VIT_iOS_Encoder_v3.mlpackage")

Running MIL default pipeline:  13%|█▎        | 10/78 [00:00<00:02, 30.24 passes/s]/usr/local/lib/python3.10/dist-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:238: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/usr/local/lib/python3.10/dist-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:266: UserWarning: Output, '1271', of the source model, has been renamed to 'var_1271' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 60.03 passes/s]


You can download the model file much faster by moving it to google drive and downloaind from there.

In [20]:
!zip -r VIT_iOS_Encoder_v3.zip VIT_iOS_Encoder_v3.mlpackage
!cp VIT_iOS_Encoder_v3.zip /content/drive/MyDrive/

  adding: VIT_iOS_Encoder_v3.mlpackage/ (stored 0%)
  adding: VIT_iOS_Encoder_v3.mlpackage/Manifest.json (deflated 60%)
  adding: VIT_iOS_Encoder_v3.mlpackage/Data/ (stored 0%)
  adding: VIT_iOS_Encoder_v3.mlpackage/Data/com.apple.CoreML/ (stored 0%)
  adding: VIT_iOS_Encoder_v3.mlpackage/Data/com.apple.CoreML/model.mlmodel (deflated 89%)
  adding: VIT_iOS_Encoder_v3.mlpackage/Data/com.apple.CoreML/weights/ (stored 0%)
  adding: VIT_iOS_Encoder_v3.mlpackage/Data/com.apple.CoreML/weights/weight.bin (deflated 7%)


## Convert Decoder

### Define structure of the decoder and load it with weights

In [22]:
def create_positional_encoding(max_length, d_model):
  pos = torch.arange(0, max_length).unsqueeze(1) # (max_length, 1)
  pos_expanded = pos.repeat(1, d_model // 2) # (max_length, d_model // 2)
  power = torch.arange(0, d_model, 2).float() / d_model
  div_term = torch.pow(10000, power).unsqueeze(0) # (1, d_model // 2)
  div_term_expanded = div_term.repeat(max_length, 1)  # (max_length, d_model // 2)

  pe = torch.zeros(max_length, d_model) # (max_length, d_model)
  pe[:, 0::2] = torch.sin(pos_expanded / div_term_expanded) # (max_length, d_model // 2)
  pe[:, 1::2] = torch.cos(pos_expanded / div_term_expanded) # (max_length, d_model // 2)

  return pe

In [23]:
class Embedding(nn.Module):
  def __init__(self, vocab_size, max_length, d_model):
    super(Embedding, self).__init__()
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=d_model)
    self.pos_encoding = create_positional_encoding(max_length, d_model).to(device)

  def forward(self, x):
    """ Apply embedding and positional encoding to the input

    Input:
      x: (N, seq_length)
    Output:
      x: (N, seq_length, d_model)
    """
    # apply embedding
    x = self.embedding(x)
    # apply positional encoding
    x += self.pos_encoding[:x.size(1)]
    return x

In [24]:
mask_size = 32
causal_mask = torch.zeros(mask_size, mask_size)
for i in range(mask_size):
    for j in range(mask_size):
        if i < j:
            causal_mask[i, j] = float('-inf')


causal_mask = causal_mask.to(device)
print(causal_mask)

tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


In [25]:
class CausalSelfAttention(nn.Module):
  def __init__(self, d_model, num_heads, dropout):
    super(CausalSelfAttention, self).__init__()
    self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=dropout, batch_first=True)
    self.layer_norm = nn.LayerNorm(d_model)


  def forward(self, x):
    attn_output, _ = self.mha(query=x, key=x, value=x, attn_mask=causal_mask, is_causal=True)
    x = self.layer_norm(x + attn_output)
    return x

In [26]:
class CrossAttention(nn.Module):
  def __init__(self, d_model, num_heads, dropout):
    super(CrossAttention, self).__init__()
    self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=dropout, batch_first=True)
    self.layer_norm = nn.LayerNorm(d_model)


  def forward(self, x, image_embedding):
    attn_output, _ = self.mha(query=x, key=image_embedding, value=image_embedding)
    x = self.layer_norm(x + attn_output)
    return x

In [27]:
class FeedForward(nn.Module):
  def __init__(self, d_model, dropout):
    super(FeedForward, self).__init__()
    self.seq = nn.ModuleList([
        nn.Linear(d_model, 2 * d_model),
        nn.ReLU(),
        nn.Linear(2 * d_model, d_model),
        nn.Dropout(dropout)
    ])

    self.layernorm = nn.LayerNorm(d_model)

  def forward(self, x):
    original_x = x
    for layer in self.seq:
      x = layer(x)
    out = x + original_x
    return self.layernorm(out)

In [28]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, dropout):
    super(DecoderLayer, self).__init__()
    self.self_attention = CausalSelfAttention(d_model=d_model, num_heads=num_heads, dropout=dropout)
    self.cross_attention = CrossAttention(d_model=d_model, num_heads=num_heads, dropout=dropout)
    self.ff = FeedForward(d_model=d_model, dropout=dropout)

  def forward(self, inputs):
    x, image_embedding = inputs
    x = self.self_attention(x)
    x = self.cross_attention(x, image_embedding)
    x = self.ff(x)
    return x

In [29]:
class Lastlayer(nn.Module):
  def __init__(self, units, vocab_size):
    super(Lastlayer, self).__init__()
    self.linear = nn.Linear(units, vocab_size)

  def forward(self, x):
    x = self.linear(x)
    return x

In [30]:
class Captioner(nn.Module):
  def __init__(self, vocab_size, max_length, d_model, num_heads, num_layers, dropout):
    super(Captioner, self).__init__()
    self.seq_embedding = Embedding(vocab_size, max_length, d_model)
    self.decoder_layers = nn.ModuleList([
    DecoderLayer(d_model, num_heads, dropout) for _ in range(num_layers)
    ])
    self.last_layer = Lastlayer(d_model, vocab_size)

    self.linear = nn.Linear(768, d_model)

  def forward(self, image_embeddings, txt):
    """
    inputs:
      txt: (N, max_length)
      image_embeddings: (N, 7, 7, 576)
    """
    x = self.seq_embedding(txt) # x: (N, max_length, d_model)
    # transform image_embeddings to proper shape
    image_embeddings = self.linear(image_embeddings)

    for dec_layer in self.decoder_layers:
      x = dec_layer((x, image_embeddings))

    # x: (N, max_length, units)

    x = self.last_layer(x) # (N, max_length, vocab_size)

    return x

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = Captioner(vocab_size = tokenizer.vocab_size, max_length = 32, d_model = 512, num_heads = 8, num_layers=4, dropout = 0.4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [32]:
checkpoint_path = '/content/drive/MyDrive/checkpoint_epoch_0_batch_2400.pth'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

<All keys matched successfully>

### Convert Decoder to CoreML model

1. Create a wrapped model.

The wrapped model will only apply softmax to output of decoder, the process of slicing and picking index with highest element will happen on Swift.

In [33]:
class WrappedDecoder(nn.Module):
  def __init__(self, decoder):
    super().__init__()
    self.decoder = decoder

  def forward(self, image_embeddings, txt):
    logit = self.decoder(image_embeddings, txt)
    pred = F.softmax(logit, dim=-1)
    return pred

In [36]:
ios_decoder = WrappedDecoder(model).eval()
ios_decoder = ios_decoder.to(device)

2. Trace the model with example inputs

In [37]:
example_txt_input = torch.randint(low=1, high=30000, size=(1, 32)).to(device)
example_image_embedding = torch.rand(1, 32, 768).to(device)
traced_decoder_model = torch.jit.trace(ios_decoder, (example_image_embedding, example_txt_input))

3. convert the traced model into a CoreML model and save it

In [38]:
decoder_model = ct.convert(
    traced_decoder_model,
    convert_to="mlprogram",
    inputs=[
        ct.TensorType(shape=example_image_embedding.shape),
        ct.TensorType(shape=example_txt_input.shape),
    ]
)

Running MIL default pipeline:  10%|█         | 8/78 [00:00<00:00, 77.55 passes/s]/usr/local/lib/python3.10/dist-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:266: UserWarning: Output, '783', of the source model, has been renamed to 'var_783' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 110.38 passes/s]


In [39]:
decoder_model.save("iOS_Decoder_V14.mlpackage")

You can download the model file much faster by moving it to google drive and downloaind from there.

In [40]:
!zip -r iOS_Decoder_V14.zip iOS_Decoder_V14.mlpackage
!cp iOS_Decoder_V14.zip /content/drive/MyDrive/

  adding: iOS_Decoder_V14.mlpackage/ (stored 0%)
  adding: iOS_Decoder_V14.mlpackage/Manifest.json (deflated 60%)
  adding: iOS_Decoder_V14.mlpackage/Data/ (stored 0%)
  adding: iOS_Decoder_V14.mlpackage/Data/com.apple.CoreML/ (stored 0%)
  adding: iOS_Decoder_V14.mlpackage/Data/com.apple.CoreML/model.mlmodel (deflated 87%)
  adding: iOS_Decoder_V14.mlpackage/Data/com.apple.CoreML/weights/ (stored 0%)
  adding: iOS_Decoder_V14.mlpackage/Data/com.apple.CoreML/weights/weight.bin (deflated 8%)


## Create vocab file

We need to build index to word json vocab file, so that we can decode tokenized array back to human readale string in our iOS app.

In [41]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab = tokenizer.get_vocab()
with open('index_to_word.json', 'w') as file:
  json.dump({idx: token for token, idx in vocab.items()}, file)